In [ ]:
import os
import time
import random
import sys

suits = ['♠', '♥', '♦', '♣']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def get_value(self):
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        return int(self.rank)

    def __repr__(self):
        return f"{self.rank}{self.suit}"

class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []
        self.money = 100
        self.bet = 0
        self.folded = False

    def hand_value(self):
        value = sum(card.get_value() for card in self.hand)
        aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and aces:
            value -= 10
            aces -= 1
        return value

class Board:
    def __init__(self):
        self.players = self.create_players()
        self.pot = 0
        self.deck = [Card(rank, suit) for suit in suits for rank in ranks]
        random.shuffle(self.deck)
        self.deal()

    def create_players(self):
        players = []
        while True:
            try:
                num = int(input("How many players are playing? "))
                if 2 <= num <= 5:
                    break
                else:
                    print("The number of players must be between 2 and 5!")
            except ValueError:
                print("Enter a valid number.")
        for i in range(1, num + 1):
            name = f"Player {i}"
            players.append(Player(name))
        print(f"\nThere are {num} players.")
        self.clear_screen()
        return players

    def clear_screen(self):
        time.sleep(0.75)
        print("\nSCREEN CLEARING...")
        time.sleep(1.75)
        try:
            from IPython.display import clear_output
            clear_output(wait=True)
        except ImportError:
            pass
        if os.name == 'nt':
            os.system('cls')
        else:
            os.system('clear')

    def deal(self):
        for player in self.players:
            player.hand = [self.deck.pop(), self.deck.pop()]

    def collect_bets(self):
        self.pot = 0
        max_bet = 0
        for player in self.players:
            player.bet = 0
            player.folded = False

        for player in self.players:
            if player.money <= 0:
                print(f"{player.name} has no money left and is out.")
                player.folded = True

        for player in self.players:
            if player.folded:
                continue
            while True:
                try:
                    print(f"\n{player.name}, you have ${player.money}.")
                    print(f"Current highest bet: ${max_bet}")
                    bet = int(input("Enter your bet (or enter less to go all-in): "))
                    if bet > player.money:
                        print("You can't bet more than you have.")
                        continue
                    player.bet = bet
                    player.money -= bet
                    self.pot += bet
                    max_bet = max(max_bet, bet)
                    break
                except ValueError:
                    print("Invalid input.")

        for player in self.players:
            if player.folded or player.bet == max_bet:
                continue
            while player.bet < max_bet:
                print(f"\n{player.name}, you need to match the highest bet of ${max_bet}. You have ${player.money} left.")
                required = max_bet - player.bet
                if player.money < required:
                    choice = input("You don't have enough to match. Do you want to go all-in or fold? (all-in/fold): ").strip().lower()
                    if choice == 'fold':
                        player.folded = True
                        break
                    elif choice == 'all-in':
                        self.pot += player.money
                        player.bet += player.money
                        player.money = 0
                        break
                    else:
                        print("Invalid input.")
                else:
                    choice = input("Do you want to match the bet or fold? (match/fold): ").strip().lower()
                    if choice == 'fold':
                        player.folded = True
                        break
                    elif choice == 'match':
                        player.money -= required
                        player.bet = max_bet
                        self.pot += required
                        break
                    else:
                        print("Invalid input.")
        self.clear_screen()

    def play_round(self):
        for player in self.players:
            if player.folded:
                continue
            while True:
                print(f"\n{player.name}'s turn!")
                print(f"Your hand: {player.hand} (value: {player.hand_value()})")
                action = input("Do you want to 'hit' or 'stand'? ").lower()
                if action == 'hit':
                    player.hand.append(self.deck.pop())
                    if player.hand_value() > 21:
                        print(f"Bust! Your hand: {player.hand} (value: {player.hand_value()})")
                        self.clear_screen()
                        break
                elif action == 'stand':
                    self.clear_screen()
                    break
                else:
                    print("Invalid input.")

    def check_winner(self):
        still_in = [(p, p.hand_value()) for p in self.players if not p.folded and p.hand_value() <= 21]
        if not still_in:
            print("\nAll players busted or folded. No winner!")
            return
        max_score = max(score for _, score in still_in)
        winners = [p for p, score in still_in if score == max_score]
        winnings = self.pot // len(winners)
        for winner in winners:
            winner.money += winnings
            print(f"\n{winner.name} wins ${winnings} with a score of {winner.hand_value()}!")
        self.pot = 0

    def show_money(self):
        for player in self.players:
            print(f"{player.name}: ${player.money}")

    def reset_hands(self):
        self.deck = [Card(rank, suit) for suit in suits for rank in ranks]
        random.shuffle(self.deck)
        self.deal()
        for player in self.players:
            player.folded = False

            player.bet = 0

    def game_loop(self):
        while True:
            self.collect_bets()
            self.play_round()
            self.check_winner()
            self.show_money()
            again = input("Do you want to play another round?").lower()
            if again != 'yes':
                break
            self.reset_hands()
            self.clear_screen()

game = Board()
game.game_loop()


Player 3 wins by the 5 card rule!


SystemExit: 

c:\Users\marti\AppData\Local\Programs\Python\Python313\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
